In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_00'
    dataset_name = 'RNA_DatasetBaseline'
    
    model_name = 'RNA_ModelV2'
    dim = 192
    depth = 12
    dim_head = 32
   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [4]:
df = pd.read_parquet(os.path.join(CFG.path,'train_data.parquet'))
ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train',sn_train=False)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train', mask_only=True,sn_train=False)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

0

In [5]:
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

epoch,train_loss,valid_loss,mae,time
0,0.141563,0.172369,0.184915,12:35
1,0.129972,0.150692,0.162133,12:38
2,0.127709,0.146774,0.158559,12:41
3,0.125981,0.143682,0.153876,12:42
4,0.124120,0.140382,0.151655,12:47
5,0.123839,0.139684,0.151146,12:54
6,0.121443,0.136980,0.147847,12:49
7,0.121307,0.136272,0.146195,12:46
8,0.121857,0.135229,0.146503,12:45
9,0.122100,0.134196,0.144159,12:45


Better model found at epoch 0 with mae value: 0.1849148832085437.
Better model found at epoch 1 with mae value: 0.16213285260299304.
Better model found at epoch 2 with mae value: 0.15855880367048467.
Better model found at epoch 3 with mae value: 0.15387598454199972.
Better model found at epoch 4 with mae value: 0.15165474721117744.
Better model found at epoch 5 with mae value: 0.15114614004350466.
Better model found at epoch 6 with mae value: 0.14784732370151515.
Better model found at epoch 7 with mae value: 0.14619540579062687.
Better model found at epoch 9 with mae value: 0.1441586461612974.
Better model found at epoch 10 with mae value: 0.14295794240033233.
Better model found at epoch 12 with mae value: 0.14153789136673883.
Better model found at epoch 14 with mae value: 0.14118654080557932.
Better model found at epoch 16 with mae value: 0.14059148864234827.
Better model found at epoch 17 with mae value: 0.14045175757502845.
Better model found at epoch 18 with mae value: 0.1399503679

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 28 with mae value: 0.1366399169735908.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 37 with mae value: 0.13417857009802886.
Better model found at epoch 45 with mae value: 0.13287746847651594.
Better model found at epoch 46 with mae value: 0.13242732530278883.
Better model found at epoch 50 with mae value: 0.1313885397826614.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 53 with mae value: 0.1309217322048733.
Better model found at epoch 54 with mae value: 0.1306941113723947.
Better model found at epoch 55 with mae value: 0.13062893894042876.
Better model found at epoch 56 with mae value: 0.13024767787506497.
Better model found at epoch 57 with mae value: 0.13024763136117495.
Better model found at epoch 58 with mae value: 0.1301469598641221.
Better model found at epoch 59 with mae value: 0.13012802657280023.
Better model found at epoch 60 with mae value: 0.13000361831189855.
Better model found at epoch 61 with mae value: 0.12991142513085738.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
rnacomp.dataset

<module 'rnacomp.dataset' from '/opt/slh/rna/exp/../rnacomp/dataset.py'>